In [1]:
import os
import sys
import math as m
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

plt.style.use('ggplot')


In [2]:
## Import data for Chengdu
weibo_CD_pt = pd.read_csv("/Users/wenfeixu/Dropbox (MIT)/MIT/CDDL/STL_WeChatProject/0_data/Weibo/Chengdu/chengdu_poi_20151213.csv",encoding="utf-8")
weibo_CD = pd.read_csv("/Users/wenfeixu/Dropbox (MIT)/MIT/CDDL/STL_WeChatProject/1_ShapeFiles/Weibo/Weibo_CD_landscanaggregated_gcj02.csv",encoding="utf-8")
landscan_CD = pd.read_csv("/Users/wenfeixu/Dropbox (MIT)/MIT/CDDL/STL_WeChatProject/1_ShapeFiles/China_LandScan2014/GCJ02/chengdu_landscan_2014_corrected_gcj02.csv",encoding="utf-8")
# LS = pd.read_csv(")


/usr/local/Cellar/python/2.7.8_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/io/parsers.py:1139: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
## Make sure everything imported correctly for Weibo Data
print weibo_CD.columns
weibo_CD.head()

Index([u'FID_1', u'pointid', u'grid_code', u'population', u'distance_m', u'Count_', u'Sum_catego', u'Sum_checki', u'Sum_chec_1', u'Sum_city', u'Sum_dianpi', u'Sum_distan', u'Sum_distri', u'Sum_enterp', u'Sum_hereno', u'Sum_lat', u'Sum_lon', u'Sum_photo_', u'Sum_pintu', u'Sum_postco', u'Sum_provin', u'Sum_sale', u'Sum_select', u'Sum_spent', u'Sum_tip_nu', u'Sum_todo_n', u'Sum_weibo_'], dtype='object')


,FID_1,pointid,grid_code,population,distance_m,Count_,Sum_catego,Sum_checki,Sum_chec_1,Sum_city,...,Sum_photo_,Sum_pintu,Sum_postco,Sum_provin,Sum_sale,Sum_select,Sum_spent,Sum_tip_nu,Sum_todo_n,Sum_weibo_
0,0,10753007,305,305,57398.170929,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,10753008,168,168,56771.422870,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,10753006,33,33,58029.179006,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,10753044,202,202,38723.238920,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,10753043,200,200,39050.786405,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
## Make sure everything imported correctly for LandScan data
print landscan_CD.columns
landscan_CD.head()

Index([u'pointid', u'grid_code', u'population', u'distance_m'], dtype='object')


,pointid,grid_code,population,distance_m
0,10753007,305,305,57398.170929
1,10753008,168,168,56771.422870
2,10753006,33,33,58029.179006
3,10753044,202,202,38723.238920
4,10753043,200,200,39050.786405


In [5]:
## Drop unnecessary columns in dataframe and give columns more accurate names for WEIBO

weibo_CD = weibo_CD.drop([ u'population', u'distance_m', u'Sum_catego',\
                          u'Sum_chec_1', u'Sum_city', u'Sum_dianpi', u'Sum_distan', u'Sum_distri',\
                          u'Sum_enterp', u'Sum_hereno', u'Sum_lat', u'Sum_lon', u'Sum_photo_', u'Sum_pintu', u'Sum_postco',\
                          u'Sum_provin', u'Sum_sale', u'Sum_select', u'Sum_spent', u'Sum_tip_nu', u'Sum_todo_n', u'Sum_weibo_',\
                          ],axis = 1)


weibo_CD.columns = ['FID','ID','grid_code','count_POIs','sum_checkin']
weibo_CD.head()



,FID,ID,grid_code,count_POIs,sum_checkin
0,0,10753007,305,0,0
1,1,10753008,168,0,0
2,2,10753006,33,0,0
3,3,10753044,202,0,0
4,4,10753043,200,0,0


In [6]:
## Drop unnecessary columns in dataframe and give columns more accurate names for LANDSCAN

landscan_CD = landscan_CD.drop(['distance_m'],axis = 1)
landscan_CD.columns = ['ID','grid_code','pop_count']
landscan_CD.head()



,ID,grid_code,pop_count
0,10753007,305,305
1,10753008,168,168
2,10753006,33,33
3,10753044,202,202
4,10753043,200,200


In [7]:
## Histogram of Logged Landscan count

fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist([m.log(x+1) for x in landscan_CD.pop_count],bins=30)


ax.set_xlabel('Grid IDs from Landcsan')
ax.set_ylabel('Normalized count on grid')
ax.set_title('Logged Landscan counts per grid')
plt.tight_layout()
plt.show()

/Library/Python/2.7/site-packages/matplotlib-override/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


In [8]:
## Histogram of Logged Weibo count

fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist([m.log(x+1) for x in weibo_CD.sum_checkin],bins=30)
# ax.hist(weibo_SH.sum_checkin,bins=20)



ax.set_xlabel('Grid IDs from Landcsan')
ax.set_ylabel('Normalized count on grid')
ax.set_title('Logged Weibo counts per grid')
plt.tight_layout()
plt.show()

In [13]:
##  Show scatter plots of original weibo and landscan side by side


fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(landscan_CD.ID, [x/float(max(landscan_CD.pop_count)) for x in landscan_CD.pop_count],marker=',',c='r',s =18)
ax.scatter(weibo_CD.ID, [x/float(max(weibo_CD.sum_checkin)) for x in weibo_CD.sum_checkin],marker = 'o',c='b',s = 18)

ax.set_xlabel('ID')
ax.set_ylabel('Frequency')
ax.set_title('Raw Weibo and Landscan Counts by ID ')
plt.tight_layout()
plt.show()


# plt.scatter(weibo_SH.ID, weibo_SH.sum_checkin)
# plt.show()

In [10]:
##  Show scatter plots of z-scores of weibo and landscan
from scipy import stats


fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(landscan_CD.ID, stats.zscore(landscan_CD.pop_count),marker=',',c='r',s =18)
ax.scatter(weibo_CD.ID, stats.zscore(weibo_CD.sum_checkin),marker = 'o',c='b',s = 18)

ax.set_xlabel('ID')
ax.set_ylabel('Z-Score')
ax.set_title('Scatter Plot of Z-Scores for Weibo and Landscan')
plt.tight_layout()
plt.show()


# plt.scatter(weibo_SH.ID, weibo_SH.sum_checkin)
# plt.show()

In [25]:
# ## Test plot with colormapping

# landscan_SH_stddev = [(x - np.mean(landscan_SH.pop_count))/float(np.std(landscan_SH.pop_count)) for x in landscan_SH.pop_count]
# landscan_SH_stddev_NORM = [(x - min(landscan_SH_stddev))/float(max(landscan_SH_stddev)-min(landscan_SH_stddev)) for x in landscan_SH_stddev]

# cm = plt.get_cmap('hsv')
# plt.scatter(landscan_SH.ID, landscan_SH.pop_count,marker=',', c=landscan_SH_stddev_NORM, cmap=cm,s =8)

# plt.colorbar()
# plt.show()

KeyboardInterrupt: 

In [15]:
## Correlation analysis for raw data
corr_CD = np.corrcoef(landscan_CD.sort('ID', ascending =True).pop_count, weibo_CD.sort('ID', ascending =True).sum_checkin)
print corr_CD

[[ 1.          0.42642906]
 [ 0.42642906  1.        ]]


In [16]:
## Correlation analysis for LOGGED data
corr_CD = np.corrcoef([m.log(x+1) for x in landscan_CD.sort('ID', ascending =True).pop_count], [m.log(x+1) for x in weibo_CD.sort('ID', ascending =True).sum_checkin])
print corr_CD

[[ 1.          0.49547076]
 [ 0.49547076  1.        ]]


In [21]:
# ## Create joined dataset 
# joined_SH = pd.merge(landscan_SH.sort('ID', ascending =True), weibo_SH.sort('ID', ascending =True), how='inner')

# ## Find correlation of just non-zero grids
# corr_SH_new = np.corrcoef(joined_SH[joined_SH.sum_checkin !=0].pop_count, joined_SH[joined_SH.sum_checkin !=0].sum_checkin)
# print corr_SH_new

[[ 1.          0.45161952]
 [ 0.45161952  1.        ]]


In [17]:
## Find "subtraction" between Landscan and Weibo z-scores of LOGGED counts

fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(landscan_CD.ID,  stats.zscore([m.log(x+1) for x in landscan_CD.pop_count])- stats.zscore([m.log(x+1) for x in weibo_CD.sum_checkin]),marker=',',c='r',s =18)

ax.set_xlabel('Grid IDs from Landcsan')
ax.set_ylabel('Diff of Normalized Counts')
ax.set_title('"Subtraction" between Weibo and Landscan Z-Scores counts by grid')
plt.tight_layout()
plt.show()



In [12]:
## Export Results of Z-score and subtraction analysis

results_CD = landscan_CD
results_CD['weibo_count'] = weibo_CD['sum_checkin']
results_CD['zscore_Landscan']=stats.zscore(landscan_CD.pop_count)
results_CD['zscore_ln_Landscan']=stats.zscore([m.log(x+1) for x in landscan_CD.pop_count])
results_CD['zscore_Weibo']=stats.zscore([m.log(x+1) for x in weibo_CD.sum_checkin])
results_CD['zscore_ln_Weibo']=stats.zscore(weibo_CD.sum_checkin)


results_CD['diff'] = results_CD['zscore_Landscan']-results_CD['zscore_Weibo']
results_CD['diff_ln'] = results_CD['zscore_ln_Landscan']-results_CD['zscore_ln_Weibo']
results_CD.head()

results_CD.to_csv('/Users/wenfeixu/Dropbox (MIT)/MIT/CDDL/STL_WeChatProject/0_data/Weibo_Landscan_Analysis/weibo_landscan_results_CD.csv',encoding="utf-8")

In [22]:
# ##  Show scatter plots of weibo and landscan side by side with only non-zero Weibo checkins

# # plt.scatter(weibo_SH.ID,[a/max(weibo_SH.sum_checkin) for a in weibo_SH.sum_checkin])

# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.scatter(joined_SH[joined_SH.sum_checkin !=0].ID, [(x - np.mean(joined_SH[joined_SH.sum_checkin !=0].pop_count))\
#                                                      /float(np.std(joined_SH[joined_SH.sum_checkin !=0].pop_count)) for x in \
#                                                      joined_SH[joined_SH.sum_checkin !=0].pop_count]\
# ,marker=',',c='r',s =14)
# ax.scatter(joined_SH[joined_SH.sum_checkin !=0].ID, [(x- np.mean(joined_SH[joined_SH.sum_checkin !=0].sum_checkin))\
#                                                       /float(np.std(joined_SH[joined_SH.sum_checkin !=0].sum_checkin)) for x in\
#                                                       joined_SH[joined_SH.sum_checkin !=0].sum_checkin]\
#                                                       ,marker = 'o',c='b',s = 14)

# ax.set_xlabel('Grid IDs from Landcsan')
# ax.set_ylabel('Normalized count on grid')
# ax.set_title('Weibo and Landscan counts per grid')
# plt.tight_layout()
# plt.show()

